<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
# Introduction to Databases: Update, Views, Inheritance, Triggers, Procedures

## Some Helper Code Used in Cells in the Notebook


In [1]:
import dbhelper

__NOTE: Show the class the new connection code or they will be miserable, i.e. closing connections!!!__

In [2]:
dbhelper.run_and_print_q("Testing helper import.","SELECT * FROM Batting WHERE playerID='willite01' and yearID='1946'",True)

Execution query = 
 SELECT * FROM Batting WHERE playerID='willite01' and yearID='1946'

 Testing helper import. 
     playerID  yearID  stint teamID lgID    G   AB    R    H  2B ...   RBI  SB  \
0  willite01    1946      1    BOS   AL  150  514  142  176  37 ...   123   0   

   CS   BB  SO  IBB  HBP SH SF GIDP  
0   0  156  44         2  0      12  

[1 rows x 22 columns]


## Questions, Comments, Discussion

## INSERT, DELETE, UPDATE

### Overview

- All database/data management systems have verbs of some form for
    - Create
    - Retrieve
    - Update
    - Delete


- Relational Theory
    - Uses selection ($\sigma$) and projection ($\pi$) for retrieve. The result is _always_ a new, derived table even if a single row is the result.
    - There are no operators for Create, Update, Delete. You can mimic the operations by deriving new tables using $\cup,$ $\cap,$ and $-.$ 


- The SQL verbs are:
    - INSERT
    - UPDATE
    - DELETE
    
    
- The REST/HTTP verbs are:
    - POST
    - PUT (or PATCH)
    - DELETE

### Setup

- To make the in lectures queries and examples simpler, we are going to use a new schema. We will continue to use Lahman2016 for "real" examples and projects.


- The initial tables/resources are: _Person, Student_ and _Faculty._

| <img src="../images/L7_inheritance_1.jpeg"> |
| --- |
| __Initial University Conceptual Model__ |


- The CREATE table for Person is

```
CREATE TABLE `person` (
  `uni` varchar(32) NOT NULL,
  `last_name` varchar(45) DEFAULT NULL,
  `first_name` varchar(45) DEFAULT NULL,
  `email` varchar(45) GENERATED ALWAYS AS (concat(`uni`,'@columbia.edu')) STORED,
  PRIMARY KEY (`uni`),
  UNIQUE KEY `email_UNIQUE` (`email`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
```

__Notes:__
- <font color=red>The database engine _always_ generates email from the uni with the concat statement. Generated fields can be _virtual_ or _stored._</font>
    - Virtual fields are computed on demand and can appear in queries.
    - Stored fields are computed but saved.
    - Virtual fields do not take up storage.
    - Stored fields take up storage and are _indexable._
    
    
- _uni_ is a primary key and must be unique. How will we accomplish this? Rely on users or application programmers to always get right?

### INSERT
#### Overview

```
INSERT [LOW_PRIORITY | DELAYED | HIGH_PRIORITY] [IGNORE]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [(col_name [, col_name] ...)]
    {VALUES | VALUE} (value_list) [, (value_list)] ...
    [ON DUPLICATE KEY UPDATE assignment_list]

INSERT [LOW_PRIORITY | DELAYED | HIGH_PRIORITY] [IGNORE]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    SET assignment_list
    [ON DUPLICATE KEY UPDATE assignment_list]

INSERT [LOW_PRIORITY | HIGH_PRIORITY] [IGNORE]
    [INTO] tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [(col_name [, col_name] ...)]
    SELECT ...
    [ON DUPLICATE KEY UPDATE assignment_list]

value:
    {expr | DEFAULT}

value_list:
    value [, value] ...

assignment:
    col_name = value

assignment_list:
    assignment [, assignment] ...
```

- Once again, this is quite a statement. We will not worry about most of the options and variations.


- The best way to learn is practice.

#### Basic INSERT Example


In [12]:
q = "INSERT INTO University.person (last_name, first_name) "
q += "VALUES('Ferguson', 'Donald');"
dbhelper.run_and_print_q("The INSERT is ",q,False)
dbhelper.run_and_print_q("The result of the insert was ","SELECT * FROM University.person;",True)

Execution query = 
 INSERT INTO University.person (last_name, first_name) VALUES('Ferguson', 'Donald');

 The INSERT is  
 True
Execution query = 
 SELECT * FROM University.person;

 The result of the insert was  
       uni last_name first_name                email
0  ferdo1  Ferguson     Donald  ferdo1@columbia.edu


#### Maybe not so Basic.

- The syntax is ```INSERT INTO TABLE (c1, c2, ...) VALUES (v1, v2, ...)```


- I did not specific a uni or email but both got set.
    - Email is not surprising, because derived from uni.
    - How did the uni get set?
    

- Let try explicitly setting the uni to "Foo"
    

In [13]:
q = "INSERT INTO University.person (uni, last_name, first_name) "
q += "VALUES('Foo','Smith', 'John');"
dbhelper.run_and_print_q("The INSERT is ",q,False)
dbhelper.run_and_print_q("The result of the insert was ","SELECT * FROM University.person;",True)

Execution query = 
 INSERT INTO University.person (uni, last_name, first_name) VALUES('Foo','Smith', 'John');

 The INSERT is  
 True
Execution query = 
 SELECT * FROM University.person;

 The result of the insert was  
       uni last_name first_name                email
0  ferdo1  Ferguson     Donald  ferdo1@columbia.edu
1  smijo1     Smith       John  smijo1@columbia.edu


- Hmm. Seemed to ignore the UNI and computed based on last_name and first_name.


- Well, __that is a good thing__ because it prevents users or dozens of programmers from having to figure out how to generate a unique uni. But how did this happen?


#### Complex INSERT Statement

- There is some error/noise in the baseball database. There are playerIDs in _Salaries_ that are not in _Master._


In [15]:
q = "SELECT * FROM Salaries WHERE "
q += " NOT EXISTS (select playerid from master where master.playerid = salaries.playerid);"
dbhelper.run_and_print_q("Orphaned playerIDs is Salaries", q, True)

Execution query = 
 SELECT * FROM Salaries WHERE  NOT EXISTS (select playerid from master where master.playerid = salaries.playerid);

 Orphaned playerIDs is Salaries 
    yearID teamID lgID   playerID    salary
0    2016    ATL   NL  pierza.01   3000000
1    2016    BOS   AL  castiru02  11400000
2    2016    COL   NL   rosajo01  12500000
3    2016    HOU   AL  harriwi10    525500
4    2016    LAD   NL  montafr02    510000
5    2016    NYY   AL  sabatc.01  25000000
6    2016    NYY   AL  willima10    509700
7    2016    TOR   AL  dicker.01  12000000


- This query detects the orphan tuples

```
SELECT
	*
FROM
	Salaries
WHERE 
	NOT EXISTS (select playerid from master where master.playerid = salaries.playerid);
```

- The basic logic is
    1. For each tuple on Salaries
    2. Find the corresponding playerID in Master.
    3. Add to the result set of the outer query if the Master tuples does not exist.
    
    
- This is an equivalent query.

```
SELECT
	*
FROM
	Salaries
WHERE 
	(select playerID from master where master.playerid = salaries.playerid) IS NULL;
```

- <font color=red>NULL works for columns but not tuples. If you SELECT more than one column, use EXISTS/NOT EXISTS.</font>


- ```INSERT INTO X (c1, c2, ...) SELECT s1,s2, ... FROM ...``` inserts all of the tuples in the derived, selected table into the the table X.


- This INSERT statement partially cleans up the data.

```
INSERT INTO Master (playerID, nameLast, nameFirst)
		SELECT playerID,
				"BADNAME",
				"BADNAME"
		FROM
			Salaries
		WHERE not exists (select playerid from master where master.playerid = salaries.playerid);
```

- The other useful pattern is <font color =blue>ON DUPLICATE</font> ..., which specifies what to do if the INSERT would fail because of a key conflict.


#### Basic UPDATE

#### Overview

- Single table format

```
UPDATE [LOW_PRIORITY] [IGNORE] table_reference
    SET assignment_list
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]

value:
    {expr | DEFAULT}

assignment:
    col_name = value

assignment_list:
    assignment [, assignment] ...

```

- Multiple table format

```
UPDATE [LOW_PRIORITY] [IGNORE] table_references
    SET assignment_list
    [WHERE where_condition]
```

- Do not worry about the multiple table format.


- Notice, the statement _sets the columns to the new values for all rows that meet WHERE condition._

#### Simple Example



In [21]:
q = "UPDATE lahman2016n.Master  SET nameFirst = 'BADFIRSTNAME' "
q += " WHERE nameLast = 'BADNAME';"
dbhelper.run_and_print_q("Changing names", q, False)
q = "SELECT playerID, nameLast, nameFirst FROM lahman2016n.Master WHERE nameLast='BADNAME';"
dbhelper.run_and_print_q("New tuples", q, True)

Execution query = 
 UPDATE lahman2016n.Master  SET nameFirst = 'BADFIRSTNAME'  WHERE nameLast = 'BADNAME';

 Changing names 
 True
Execution query = 
 SELECT playerID, nameLast, nameFirst FROM lahman2016n.Master WHERE nameLast='BADNAME';

 New tuples 
     playerID nameLast     nameFirst
0  castiru02  BADNAME  BADFIRSTNAME
1  dicker.01  BADNAME  BADFIRSTNAME
2  harriwi10  BADNAME  BADFIRSTNAME
3  montafr02  BADNAME  BADFIRSTNAME
4  pierza.01  BADNAME  BADFIRSTNAME
5   rosajo01  BADNAME  BADFIRSTNAME
6  sabatc.01  BADNAME  BADFIRSTNAME
7  willima10  BADNAME  BADFIRSTNAME


### DELETE

#### Overview

- Single table syntax

```
DELETE [LOW_PRIORITY] [QUICK] [IGNORE] FROM tbl_name
    [PARTITION (partition_name [, partition_name] ...)]
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]
```

- Multiple table syntax

```
DELETE [LOW_PRIORITY] [QUICK] [IGNORE]
    tbl_name[.*] [, tbl_name[.*]] ...
    FROM table_references
    [WHERE where_condition]

DELETE [LOW_PRIORITY] [QUICK] [IGNORE]
    FROM tbl_name[.*] [, tbl_name[.*]] ...
    USING table_references
    [WHERE where_condition]
```

- We will ignore multiple table syntax for now.


#### Example

```
CREATE TABLE SalariesBackup AS SELECT * FROM Salaries;

DELETE FROM Salaries WHERE yearID > 2015;
```

- The CREATE statement is a diversion. This is basically a way to make a base table from a derived table.

- The DELETE statement has the obvious meanings.

### The Magically Appearing UNI

- The code

```
CREATE DEFINER=`root`@`localhost` TRIGGER `University`.`person_BEFORE_INSERT`
	BEFORE INSERT ON `person` FOR EACH ROW
BEGIN
	SET New.uni = LOWER(generate_uni(New.last_name, New.first_name));
END

CREATE DEFINER=`root`@`localhost` TRIGGER `University`.`person_BEFORE_UPDATE`
	BEFORE UPDATE ON `person` FOR EACH ROW
BEGIN
	SET New.uni = Old.uni;
END

CREATE DEFINER=`root`@`localhost` FUNCTION `generate_uni`(last_name VARCHAR(32), 
	first_name VARCHAR(32)) RETURNS varchar(8) CHARSET utf8
BEGIN


	DECLARE		c1				CHAR(3);
    DECLARE		c2				CHAR(2);
    DECLARE		prefix			CHAR(6);
    DECLARE		uniCount		INT;
    DECLARE		newUni		VARCHAR(8);
    
    SET c1 		=		UPPER(SUBSTR(last_name, 1, 3));
    SET c2		=		UPPER(SUBSTR(first_name, 1, 2));
    SET prefix	=		CONCAT(c1, c2, "%");
    
    SET uniCount = 0;
    
    SELECT COUNT(uni) INTO uniCount FROM person WHERE uni LIKE prefix;
	

	SET newUni = CONCAT(c1, c2,uniCount+1);

RETURN  newUni;
END
```

- Walk through in MySQL Workbench.

### SQL/PSM

- Ramakrishnan and Gehrke, section 6.5.3

"SQL/PSM (SQL/Persistent Stored Modules) is an ISO standard mainly defining an extension of SQL with a procedural language for use in stored procedures." (https://en.wikipedia.org/wiki/SQL/PSM) 


- What is a stored procedure?
    - Program executed through single SQL statement
    - Executed in process space of the server


- Advantages:
    - Can encapsulate application logic while staying “close” to the data
    - Reuse of application logic by different users
    - Avoid tuple-at-a-time return of records through cursors
    - Enhance integrity:
        - Replace arbitrary data changes through INSERT, UPDATE and DELETE
        - With tested, validated operations from the DBA.
        
        
- Procedural programming appears in three contexts
    - Stored procedures.
    - Functions.
    - Triggers.
    

- Basic language constructs
    - IN, OUT, INOUT parameters.
    - Compound BEGIN ... END statement blocks.
    - DECLARE local variables.
    - SET variables to values.
    - Control flow
        - CASE 
        - IF ... THEN ... ELSE ...
        - ITERATE 
        - LEAVE
        - LOOP 
        - REPEAT
        - RETURN
        - WHILE
        
        
- Language detailes differ between database management systems, but there is a common, core standard.


### Functions

- All parameters are IN parameters.


- Functions return values.


- Functions cannot access data and schema, but cannot change data or schema.


- Function to create a uni.

```
# Compute a UNI based on user's names and already existing UNIs.
# NOTE that we do not need to declare parameters to be IN. All function parameters are IN.
CREATE DEFINER=`root`@`localhost` FUNCTION `generate_uni`(last_name VARCHAR(32), 
	first_name VARCHAR(32)) RETURNS varchar(8) CHARSET utf8
BEGIN

	# You have to/should dcelare variables before using
	DECLARE		c1				CHAR(3);
    DECLARE		c2				CHAR(2);
    DECLARE		prefix			CHAR(6);
    DECLARE		uniCount		INT;
    DECLARE		newUni		    VARCHAR(8);
    
    # UNI will be first three characters from last_name +
    # first two characters from first name + a number.
    # The number will be one more than the number of existing
    # UNIs with the same first five characters. So, our query will
    # look for strings like "FERDO%"
    SET c1 		=		UPPER(SUBSTR(last_name, 1, 3));
    SET c2		=		UPPER(SUBSTR(first_name, 1, 2));
    SET prefix	=		CONCAT(c1, c2, "%");
    
    SET uniCount = 0;
    
    # Count the number of matching UNIs.
    SELECT COUNT(uni) INTO uniCount FROM person WHERE uni LIKE prefix;
	
	# New UNI is prefix + (count + 1)
	SET newUni = CONCAT(c1, c2,uniCount+1);

RETURN  newUni;
END
```

### Triggers

Ramakrishnan and Gehrke, section 5.8

#### Overview

- Trigger: _procedure_ that starts automatically if specified changes occur to the DBMS (especially data or schema).


- Three elements:
    - Event (activates the trigger)
    - Condition (tests whether the triggers should run)
    - Action (what happens if the trigger runs)


- Most comment events and actions
    - Table/tuple data is about to change $\rightarrow$ implement complex constraint, derive values, etc.
    - Table data just changed $\rightarrow$ side effect, log information, etc.
    
    
- Create and modify triggers using DDL or tools that generate DDL.


- MySQL Workbench

| <img src="../images/L7_workbench_triggers.jpeg"> |
| --- |
| __MySQL Workbench Triggers__ |





#### First Example -- uni

- (Hypothetical) uni requirements:
    - Must be unique.
    - Must be relatively easy to remember, enter into an application, tell to someone helping you, etc. $\rightarrow$ a UUID like approach does not meet requirements. (Sample UUID = 429af6cc-8c0b-4d7b-a78a-627728c6f801)
    
    
- Generate using a simple function of new person's name and all previously registered people's UNIs.


- We could publish the function definition and assume that all programs connecting to the database will correctly implement the function, but this is risky and the function might change.


- The function is an intrinsic, _active_ property of the database.


- The trigger and procedures need to execute _before_ INSERT and UPDATE.


- The parameters to
    - An INSERT trigger is _New_ tuple.
    - An UPDATE trigger are the _New_ changed tuple and the previous _Old_ tuple. 
    
    
- The triggers can modify _New_ parameter before the update actually occurs.

```
CREATE DEFINER=`root`@`localhost` TRIGGER `University`.`person_BEFORE_INSERT`
	BEFORE INSERT ON `person` FOR EACH ROW
BEGIN
	SET New.uni = LOWER(generate_uni(New.last_name, New.first_name));
END

CREATE DEFINER=`root`@`localhost` TRIGGER `University`.`person_BEFORE_UPDATE`
	BEFORE UPDATE ON `person` FOR EACH ROW
BEGIN
	# Ignore the attempted change.
    # There are ways to abort the action, signal errors, etc.
    # But for now we just silently ignore the change.
	SET New.uni = Old.uni;
END
```
   
- There are ways to signal non-fatal warning and fatal errors that abort the operation, but we will cover later.

### Object Relational Mapping (ORM)

#### Overview

- Object oriented systems/programming have four characteristics
    - Abstraction
    - Encapsulation
    - Inheritance
    - Polymorphism
    
    
- Abstraction, encapsulation and polymorphism are principles that apply to the application.


- _Inheritance_ applies to data and is a type of _relationship_ between data types. $\rightarrow$ Must be able to model/represent in a database because "relational data" is about "relations" between entity properties and entity types.

#### A Digression on Modeling, Notation and Relationships

##### Crow's Foot Notation

- I normally use this notation in class.


- MySQL Workbench Designer also uses an extended version of this notation.

| <img src="../images/crows_foot_notation.jpeg" width="75%"> |
| --- |
| [Crow's Foot Notation](http://www.vivekmchawla.com/erd-crows-foot-relationship-symbols-cheat-sheet/) |

##### Chen Notation
- I tend to use simple entity-relationationship notation in class. This is not a data modeling course.


- Crow's Foot is simple and precise, and good enough for our purposes.


- The textbook uses Chen Notation, which is more complete and complex.

| <img src="../images/chen_notation.jpeg">
| --- |
| __Chen Notation__ |

##### Unified Modeling Language

Ramakrishnan and Gehrke, section 2.7

- There are many ways to classify ER relationships
    - Optional versus Mandatory.
    - Endpoint cardinality.
    - Logical containment (Weak Entity) versus references.
    
    
- Objected oriented adds an additional type of relationship, _Is A._
    - A student _Enrolls In_ classes. Students and classes have different schema and data elements.
    - A student _Is A_ person. A student's schema and properties is somehow linked to person.
    

- The [Unified Modeling Language (UML)](https://en.wikipedia.org/wiki/Unified_Modeling_Language) is the most common notation for object-oriented modeling.


- UML relationships

| <img src="../images/UML_relationships.jpeg"> |
| --- |
| __UML Relationships__ |

### Object Oriented and Inheritance Modeling Patterns

#### Overview

- This is an incredibly complex problem in general.


- There is an industry of products and tools for [object relational mapping (ORM).](https://en.wikipedia.org/wiki/Object-relational_mapping) 


- We are going to focus on core, simple cases because
    - The pattern is common in database design and implementation scenarios.
    - A use case that shows roles of views, constraints, triggers, functions and procedures.
    
    
- There are [three basic patterns:](http://www.vertabelo.com/blog/technical-articles/inheritance-in-a-relational-database) 1 table, 2 table and 3 table.


- The technical term for _inheritance_ and _realization_ is _specialization._


- There are two core _constraints_ or _facets_ of specialization
    - _Incomplete/complete_
        - In an incomplete specialization only some instances of the parent class are specialized (have unique attributes). Other instances of the parent class have only the common attributes.
        - In a complete specialization, every instance of the parent class has one or more unique attributes that are not common to the parent class.
    - _Disjoint/overlapping_
        - In a disjoint specialization, an object could be a member of only one specialized subclass.
        - In an overlapping specialization, an object could be a member of more than one specialized subclass.
    

#### One Table Solution

##### Overview and Views

| <img src="../images/inheritance_one.jpeg"> |
| --- |
| __One Table Solution__ |

- There is a single table containing all data for all instances of all types.


- Database views implement _projection_ to create the entity types _student_ and _faculty._

```
CREATE TABLE `person_one` (
  `uni` varchar(32) NOT NULL,
  `last_name` varchar(45) NOT NULL,
  `first_name` varchar(45) NOT NULL,
  `email` varchar(45) NOT NULL,
  `type` enum('S','F') NOT NULL,
  `title` varchar(45) DEFAULT NULL,
  `year` varchar(45) DEFAULT NULL,
  `school` varchar(45) DEFAULT NULL,
  PRIMARY KEY (`uni`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

CREATE VIEW
	faculty_one
AS SELECT
	uni, last_name, first_name, email, title
FROM
	University.person_one
WHERE
	University.person_one.type = 'F';
    
CREATE VIEW
	student_one
AS SELECT
	uni, last_name, first_name, school, year
FROM
	University.person_one
WHERE
	University.person_one.type = 'S';

```

- This approach is fine for _retrieve,_ but what about _create, update_ and _delete._
    - Programmers think in terms of create, update and delete student/faculty but need to know implementation details that may change. What is I add a new type _staff?_ This could force changing client programs.
    - There are some odd constraints. _year_ must be NULL if _type_ is _F_ but NOT NULL if type is _S._
    
    
- Our solution will be
    - Implement CREATE, UPDATE and DELETE triggers on _Person._
    - Restrict access to INSERT, UPDATE and DELETE on _Person_ to specific security roles.
    - Provide procedures _create_student, delete_student, update_student, create_faculty, delete_faculy_ and _update_faculty._
    
    
- This approach improves data integrity, simplifies client application development and isolates applications from schema and design changes.

##### Person

- Identify the more complex integrity rules. We will "protect" the person table behind procedures but still need to protect the table's integrity.

```
CREATE DEFINER=`root`@`localhost` TRIGGER `University`.`person_one_BEFORE_INSERT`
	BEFORE INSERT ON `person_one` FOR EACH ROW
BEGIN

	SET New.uni = LOWER(generate_uni(New.last_name, New.first_name));
    
    IF New.type = 'S' THEN
    
		IF (New.year IS NULL) OR (New.school is NULL) OR (New.title IS NOT NULL) THEN
			SIGNAL SQLSTATE '45001'
				SET MESSAGE_TEXT = 'Invalid properties for student', MYSQL_ERRNO=9001;
		END IF;
        
	ELSE
    
		IF (New.year IS NOT NULL) OR (New.school is NOT NULL) OR (New.title IS NULL) THEN
			SIGNAL SQLSTATE '45001'
				SET MESSAGE_TEXT = 'Invalid properties for faculty', MYSQL_ERRNO=9001;
		END IF;
		
	END IF;

END
```

- We would do a similar procedure for UPDATE.


- Now, we need to stop client applications from directly messing things up. So, we will limit the privileges for the applications.

```
REVOKE ALL ON * FROM university_client;
GRANT SELECT ON University.student_one TO university_client;
GRANT SELECT ON University.faculty_one TO university_client;
```

- The REVOKE and GRANT syntaxes are complicated, and the exact model and syntax vary between database management systems.


- The common core elements of any _role based access control rule_(https://en.wikipedia.org/wiki/Role-based_access_control) are
    - [Least privilege principle](https://en.wikipedia.org/wiki/Principle_of_least_privilege): Give a user/role ONLY the permissions necessary to implement functions. 
    - The user (role, group), e.g. citizenship, clearance, etc.
    - The resource e.g. schema/database, table, view, etc.
    - The action, e.g. INSERT, DROP TABLE, ...
    - The context e.g. time, location, source IP, ...
    

- Applied to relational (MySQL) this means

```
GRANT sql_verb(s) ON sql_resource(s) TO user_id(s)
```

##### Procedures

- There will be a set of procedures encapsulating the underlying table.


- The client applications
    - Cannot INSERT, UPDATE, DELETE tables.
    - Can CALL procedures.
    - The databasea analyst carefully develops procedures to ensure integrity. The procedures can INSERT, UPDATE and DELETE the base tables because they run with the permissions of the procedure writer.

```
DELIMITER $$
CREATE DEFINER=`root`@`localhost` 
	PROCEDURE `create_student_one`(IN last_name VARCHAR(64),
        IN first_name VARCHAR(64), IN year CHAR(4), IN school CHAR(6))
    SQL SECURITY DEFINER
BEGIN

	# Your code goes here.
    # ... ...
    
END$$
DELIMITER ;
```

- The GRANT statement

```
GRANT  EXECUTE  ON PROCEDURE University.create_student_one TO university_client;
```